<a href="https://colab.research.google.com/github/Shaka42/titanic-competition/blob/main/Binary_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import layers



In [31]:
train_dataset = pd.read_csv('train.csv')
train_dataset.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187.0,0.0
1,1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288.0,1.0
2,2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254.0,0.0
3,3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76.0,0.0
4,4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294.0,0.0


In [32]:
print(train_dataset.describe())
print(train_dataset.info())

                 id           Age  Driving_License   Region_Code  \
count  1.512721e+06  1.512721e+06     1.512721e+06  1.512721e+06   
mean   7.563600e+05  3.840413e+01     9.980519e-01  2.642938e+01   
std    4.366851e+05  1.500646e+01     4.409480e-02  1.298691e+01   
min    0.000000e+00  2.000000e+01     0.000000e+00  0.000000e+00   
25%    3.781800e+05  2.400000e+01     1.000000e+00  1.500000e+01   
50%    7.563600e+05  3.600000e+01     1.000000e+00  2.800000e+01   
75%    1.134540e+06  4.900000e+01     1.000000e+00  3.500000e+01   
max    1.512720e+06  8.500000e+01     1.000000e+00  5.200000e+01   

       Previously_Insured  Annual_Premium  Policy_Sales_Channel       Vintage  \
count        1.512721e+06    1.512721e+06          1.512720e+06  1.512720e+06   
mean         4.623602e-01    3.045833e+04          1.123400e+02  1.639143e+02   
std          4.985814e-01    1.643025e+04          5.407590e+01  7.993397e+01   
min          0.000000e+00    2.630000e+03          1.000000e+00

In [33]:
train_dataset.isna().sum()

id                      0
Gender                  0
Age                     0
Driving_License         0
Region_Code             0
Previously_Insured      0
Vehicle_Age             0
Vehicle_Damage          0
Annual_Premium          0
Policy_Sales_Channel    1
Vintage                 1
Response                1
dtype: int64

In [34]:
for column in train_dataset.columns:
  train_dataset[column]=train_dataset[column].fillna(train_dataset[column].mode()[0])

In [35]:
train_dataset['Vehicle_Age'].unique()

array(['1-2 Year', '> 2 Years', '< 1 Year'], dtype=object)

In [36]:
train_dataset=pd.get_dummies(data=train_dataset,columns=['Gender','Vehicle_Damage','Vehicle_Age'],dtype=float)

In [37]:
train_dataset=train_dataset.drop('id',axis=1)

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

features = train_dataset.drop('Response',axis=1)
Labels = train_dataset.pop("Response")

X_train, X_test, Y_train, Y_test = train_test_split(features,Labels,test_size=0.2)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [39]:
X_train_scaled.shape

(1210176, 14)

In [65]:
#Building model
model = keras.Sequential()
model.add(layers.Dense(128, activation='relu', input_shape=(14,)))
model.add(layers.Dense(128,activation='relu'))
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(2,activation='sigmoid'))

In [66]:
model.compile(optimizer='sgd',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])


In [67]:
from keras.callbacks import EarlyStopping,ModelCheckpoint

early_stopping_monitor = EarlyStopping(patience=2)


In [68]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_26 (Dense)            (None, 128)               1920      
                                                                 
 dense_27 (Dense)            (None, 128)               16512     
                                                                 
 dense_28 (Dense)            (None, 64)                8256      
                                                                 
 dense_29 (Dense)            (None, 2)                 130       
                                                                 
Total params: 26818 (104.76 KB)
Trainable params: 26818 (104.76 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [69]:
#Train model
model.fit(X_train_scaled,Y_train,epochs=10,validation_split=0.3,verbose=1,callbacks=[early_stopping_monitor])

Epoch 1/10
26473/26473 [==============================] - 105s 4ms/step - loss: 0.2694 - accuracy: 0.8771 - val_loss: 0.2691 - val_accuracy: 0.8770
Epoch 2/10
26473/26473 [==============================] - 104s 4ms/step - loss: 0.2660 - accuracy: 0.8774 - val_loss: 0.2662 - val_accuracy: 0.8770
Epoch 3/10
26473/26473 [==============================] - 102s 4ms/step - loss: 0.2655 - accuracy: 0.8774 - val_loss: 0.2665 - val_accuracy: 0.8770
Epoch 4/10
26473/26473 [==============================] - 106s 4ms/step - loss: 0.2652 - accuracy: 0.8774 - val_loss: 0.2660 - val_accuracy: 0.8771
Epoch 5/10
26473/26473 [==============================] - 105s 4ms/step - loss: 0.2650 - accuracy: 0.8774 - val_loss: 0.2659 - val_accuracy: 0.8772
Epoch 6/10
26473/26473 [==============================] - 124s 5ms/step - loss: 0.2648 - accuracy: 0.8775 - val_loss: 0.2656 - val_accuracy: 0.8771
Epoch 7/10
26473/26473 [==============================] - 104s 4ms/step - loss: 0.2647 - accuracy: 0.8774 - val_

In [70]:
loss , acc = model.evaluate(X_test_scaled,Y_test)

9455/9455 [==============================] - 23s 2ms/step - loss: 0.2636 - accuracy: 0.8777


In [178]:
test_dataset = pd.read_csv('test.csv')

In [181]:
test_dataset.duplicated().sum()

0

In [155]:
test_dataset['Vehicle_Age'].unique()

array(['< 1 Year', '1-2 Year', '> 2 Years', '1-2'], dtype=object)

In [156]:
test_dataset['Vehicle_Age'] = test_dataset['Vehicle_Age'].replace('1-2','1-2 Year')

In [157]:
test_dataset.isna().sum()

id                      0
Gender                  0
Age                     0
Driving_License         0
Region_Code             0
Previously_Insured      0
Vehicle_Age             0
Vehicle_Damage          1
Annual_Premium          1
Policy_Sales_Channel    1
Vintage                 1
dtype: int64

In [158]:
for column in test_dataset.columns:
  test_dataset[column]=test_dataset[column].fillna(test_dataset[column].mode()[0])

In [159]:
test_dataset=pd.get_dummies(data=test_dataset,columns=['Gender','Vehicle_Damage','Vehicle_Age'],dtype=float)

In [160]:
pd.read_csv('sample_submission.csv')

,id,Response
0,11504798,0.5
1,11504799,0.5
2,11504800,0.5
3,11504801,0.5
4,11504802,0.5
...,...,...
6775409,18280207,0.5
6775410,18280208,0.5
6775411,18280209,0.5
6775412,18280210,0.5


In [161]:
test_dataset=test_dataset.drop('id',axis=1)
test_scaled = scaler.fit_transform(test_dataset)

In [162]:
test_dataset

,Age,Driving_License,Region_Code,Previously_Insured,Annual_Premium,Policy_Sales_Channel,Vintage,Gender_Female,Gender_Male,Vehicle_Damage_No,Vehicle_Damage_Yes,Vehicle_Age_1-2 Year,Vehicle_Age_< 1 Year,Vehicle_Age_> 2 Years
0,20,1,47.0,0,2630.0,160.0,228.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
1,47,1,28.0,0,37483.0,124.0,123.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
2,47,1,43.0,0,2630.0,26.0,271.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,22,1,47.0,1,24502.0,152.0,115.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
4,51,1,19.0,0,34115.0,124.0,148.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1519807,38,1,29.0,1,37272.0,26.0,145.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
1519808,42,1,1.0,0,2630.0,22.0,245.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
1519809,28,1,36.0,0,2630.0,160.0,164.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
1519810,24,1,12.0,1,31090.0,152.0,198.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0


In [111]:
train_dataset

,Age,Driving_License,Region_Code,Previously_Insured,Annual_Premium,Policy_Sales_Channel,Vintage,Gender_Female,Gender_Male,Vehicle_Damage_No,Vehicle_Damage_Yes,Vehicle_Age_1-2 Year,Vehicle_Age_< 1 Year,Vehicle_Age_> 2 Years
0,21,1,35.0,0,65101.0,124.0,187.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
1,43,1,28.0,0,58911.0,26.0,288.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,25,1,14.0,1,38043.0,152.0,254.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
3,35,1,1.0,0,2630.0,156.0,76.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
4,36,1,15.0,1,31951.0,152.0,294.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1512716,37,1,17.0,0,2630.0,124.0,210.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
1512717,35,1,18.0,1,33045.0,26.0,71.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
1512718,24,1,15.0,1,28981.0,152.0,47.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
1512719,25,1,11.0,1,30498.0,152.0,71.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0


In [163]:
test_scaled.shape

(1519812, 14)

In [164]:
predict = model.predict(test_scaled)

47495/47495 [==============================] - 79s 2ms/step


In [165]:
label = list()
for i in list(range(len(predict))):
    label.append(np.argmax(predict[i]))

In [168]:
pred = pd.DataFrame(data={'id':test_dataset['id'],'Response':label})

In [172]:
pred.to_csv('submission_2.csv')

In [173]:
pred.shape

(1519812, 2)